In [1]:
%matplotlib inline
import time
start_time = time.time()

import math
import scipy
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import pandas as pd
import seaborn as sns
from scipy.stats import norm
from scipy.stats import zscore
import scipy.interpolate as spl

my_data = np.loadtxt('ngc_1_inner_1.2')
x_data = my_data[:,24]
y_data = my_data[:,10]

outer_data = np.loadtxt('ngc_1_outer_1.5_2.7')
x_out = outer_data[:,24]
y_out = outer_data[:,10]

isochrone_data = np.loadtxt('isochrone_final')
x_iso = isochrone_data[:,2]
y_iso = isochrone_data[:,0]

plt.style.use('default')

print("********* THIS IS CMD FOR NGC 1 ************")
plt.figure()

plt.scatter(x_data, y_data, 1.0, 'black')
plt.plot(x_iso, y_iso, 1.5, 'red')
plt.xlim(0,2.8)
plt.ylim(18, 23)
plt.gca().invert_yaxis()
plt.xlabel("F475W - F814W", fontsize=20)
plt.ylabel("F475W", fontsize=20)
plt.title('CMD of NGC 1', fontsize=20)
plt.show()

print("********* THIS IS OUTER CMD FOR NGC 1 ************")
plt.figure()

plt.scatter(x_out, y_out, 1, 'black')
#plt.plot(x_iso, y_iso, 1.5, 'red')
plt.xlim(0,2.8)
plt.ylim(18, 23)
plt.gca().invert_yaxis()
plt.xlabel("F475W - F814W", fontsize=20)
plt.ylabel("F475W", fontsize=20)
plt.title('CMD of NGC 1', fontsize=20)
plt.show()

# Choose an arbitrary point on CMD 
#plt.plot(x_data[2016],y_data[2016], 'g*')  # The pale green dot. Not so pale. Okay!
plt.show()

df = pd.DataFrame({'x_data':x_data, 'y_data':y_data})
#print(df)
df_cut = df[(df['x_data']<=3) & (df['x_data']>= 1.4) & (df['y_data']<=22) & (df['y_data']>=18)]

# We get the index of each value we want to get rid of, using the method .index.to_numpy()
df_cut_index = df_cut[(df_cut.x_data < 1.6) & (df_cut.y_data < 19.9)].index.to_numpy()

# Now we use the drop method to get rid of the values located on those indexes.
df_cut_1 = df_cut.drop(df_cut_index)
#print(df_cut_index)

plt.figure()
plt.grid()
plt.scatter(df_cut.x_data, df_cut.y_data, 1, 'black')
plt.xlim(0.5,3.5)
plt.ylim(18, 22)
plt.xlabel('m_F475W - m_F814W (mag)', fontsize=15)
plt.ylabel('m_F475W (mag)', fontsize=15)
plt.title("RGB of NGC 1", fontsize=20)
plt.gca().invert_yaxis()
plt.show()



plt.figure()
plt.grid()
plt.scatter(df_cut_1.x_data, df_cut_1.y_data, 1, 'black')
plt.xlim(0.5,3.5)
plt.ylim(18, 22)
plt.xlabel('m_F475W - m_F814W (mag)', fontsize=15)
plt.ylabel('m_F475W (mag)', fontsize=15)
plt.title("RGB without Red Clump (RC)", fontsize=15)
plt.gca().invert_yaxis()
plt.show()

#print(df_cut_1)


#define a dataframe for x data and y data
df_new= pd.DataFrame({'x_data':df_cut_1.x_data,'y_data':df_cut_1.y_data})
print(len(df_new))

df1_new = df_new[(df_new['x_data']<=2.2) & (df_new['x_data']>= 1.0) & (df_new['y_data']<=22) & (df_new['y_data']>=18)]
print(len(df1_new))
bins = np.linspace(18,22, num=11) # These are number of bins

md = pd.DataFrame(np.zeros(((len(bins)-1), 2)), columns=['x_med','y_med'])
#print(md)

for i in range(len(bins)-1):
    tempdf = df1_new[(df1_new['y_data'] >= bins[i]) & (df1_new['y_data'] <= bins[i+1]) ]
    x_median = np.median(tempdf['x_data'])
    y_median = np.median(tempdf['y_data'])
    md.iloc[i]=[x_median, y_median]
    
#print(md)

plt.figure()
plt.plot(md['x_med'], md['y_med'], 'black')
plt.grid()
plt.scatter(df1_new['x_data'], df1_new['y_data'], 1, 'black', zorder=0) # zorder= is used when you are lazy. 
plt.xlim(0.5, 3.5)
plt.ylim(18, 22)
plt.xlabel('m_F475W - m_F814W (mag)', fontsize=15)
plt.ylabel('m_F475W (mag)', fontsize=15)
plt.title("Fiducial line through RGB", fontsize=20)
plt.gca().invert_yaxis() # Again. When you are lazy, use this. Being lazy is good and easy. 
plt.show()
plt.savefig('Fiducial_1.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

a = pd.cut(df1_new.y_data, bins)
b = df1_new.groupby(['y_data', a]).y_data.median()

df2 = pd.DataFrame({'values':b})
np.savetxt('cut_group_1.txt', df2)

df1_new['x_diff_new'] = df1_new.groupby(a).apply(lambda grp: grp.x_data - grp.x_data.median()).values
plt.hist(df1_new.x_diff_new, bins=90,cumulative=False,histtype='step',orientation='vertical')
plt.title('Histogram: Distance Distribution from the Fiducial Line', fontsize=12)
plt.xlabel('Pixel Distance' ,fontsize=20)
plt.ylabel('Number of Stars', fontsize=20)
plt.xlim(-0.20, 0.21)
plt.show()

points_fid = spl.interp1d(md['y_med'], md['x_med'], fill_value='extrapolate')
#print(points_fid(18))

#print(df_cut_1)

print('loading of data is completed in [{}] seconds'.format(time.time() - start_time))


OSError: ngc_1_inner_1.2 not found.

In [ ]:
sns.distplot(df1_new.x_diff_new, 33,kde_kws={"color": "red", "lw": 2.5, "label": "KDE"})
plt.xlabel('Pixel Distance', fontsize=20)
plt.ylabel('N', fontsize=20)
plt.title('Kernel Density Estimation (KDE) for NGC 1', fontsize=14)
plt.savefig('kde_1.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)